# Neural Networks

## Neural Network with TensorFlow

**Interesting Facts:**
* Deep learning library developed by Google Brain Team
* Computationaly efficient (eliminates GPU overhead by performing computations *entirely* outside of Python)
* Includes a visualization board
* Well documented

**What it does:**
* Graph as a data structure
  * Computations are represented as graphs 
  * Information is stores inside a graph (TensorFlow variables)
* Executes computations inside Session instances
  * Feeds / Fetches to retreive results

**Workflow phases:**
* Construction Phase
  * Encode data as a one dimensional vector
  * Define structure Variables
* Execution Phase
  * Initiate Interactive Session
  * Train the model
  * Retrieve results

**Tensor** is a **multi-dimensional array**. A TensorFlow tensor as an n-dimensional array or list. 

**Nodes** in the graph are called **ops** (short for operations). An op takes zero or more Tensors, performs some computation, and produces zero or more Tensors.

A TensorFlow graph is a description of computations. To compute anything, a graph must be launched in a Session. A Session places the graph ops onto Devices, such as CPUs or GPUs, and provides methods to execute them. These methods return tensors produced by ops as numpy ndarray objects in Python.

### Data Preparation

39774 of recipes, 7126 unique ingredients, 20 cuisines.

Redundant ingredients, ingredients name not standardized (brands, measurements, types)

* Every recipe and cuisine must be converted to One-Hot Vectors

* [0,0,0,1,0,0,1]

* Need two tensors, one for ingredients and one for cuisine labels

In [1]:
import json
import numpy as np
import random

In [2]:
train_file = "train.json"

with open(train_file, 'r') as fh:
    train_ln = json.load(fh)
    
test_file = "test.json"

with open(test_file, 'r') as fh:
    test_ln = json.load(fh)

In [3]:
cuisines_train = list()
ingredients_train = list()
for i in train_ln:
    cuisines_train.append(i['cuisine'])
    ingredients_train.append([i.lower() for i in i['ingredients']])
    
ingredients_test = list()
for i in test_ln:
    ingredients_test.append([i.lower() for i in i['ingredients']])    

In [4]:
print len(ingredients_train)
print len(cuisines_train)
print len(ingredients_test)

39774
39774
9944


In [5]:
# Keep track of test ids
test_ids = [i['id'] for i in test_ln]

In [6]:
# Difference between train and test sets
diff_ingr = list(set([item for sub in ingredients_train for item in sub]) - set([item for sub in ingredients_test for item in sub]))
for i in diff_ingr:
    if i == 'bear':
        print i
        
# Bear Recipe

for i in ingredients_train:
    for j in i:
        if j == 'bear':
            print i
            
print len(diff_ingr)

bear
[u'pancetta', u'vegetable oil', u'spelt flour', u'egg yolks', u'salt', u'onions', u'black pepper', u'buttermilk', u'garlic cloves', u'bear', u'all-purpose flour']
2647


In [7]:
# Get unique values for cuisine and ingredients
cuisines_set = set(cuisines_train)
ingredients_set = set([item for sub in ingredients_train for item in sub] + [item for sub in ingredients_test for item in sub])
print len(ingredients_set)

7126


In [8]:
# Create binary mapping for each ingredient and cuisine

ingredient_mapper = dict()
for count, elem in enumerate(ingredients_set):
    ingredient_mapper[elem] = count
    
cuisines_mapper = dict()
for count, elem in enumerate(cuisines_set):
    cuisines_mapper[elem] = count

# Save keys in a list
ingredients_keys = ingredient_mapper.keys()
cuisines_keys = cuisines_mapper.keys()

In [9]:
def encode_ingredient(ingredient_list):
    encoded_ingr = np.zeros(len(ingredients_set))
    for ing in ingredient_list:
        index = ingredient_mapper[ing]
        encoded_ingr[index] = 1
    return encoded_ingr

def encode_cuisine(cuisine_list):
    encoded_cuis = np.zeros(len(cuisines_set))
    if i in cuisines_keys:
        encoded_cuis[cuisines_mapper[i]] = 1
    return encoded_cuis

In [10]:
# Produce encoded 7126-dimensional vector for ingredients (train)

encoded_ingredients_list_train = list()
for i in ingredients_train:
    encoded_ingredients_list_train.append(encode_ingredient(i))

In [11]:
# Produce encoded 20-dimensional vector for cuisines

encoded_cuisines_list_train = list()
for i in cuisines_train:
    encoded_cuisines_list_train.append(encode_cuisine(i))

In [12]:
# Produce encoded 7126-dimensional vector for ingredients (test)

encoded_ingredients_list_test = list()
for i in ingredients_test:
    encoded_ingr = np.zeros(len(ingredients_set))
    for j in i:
        index = ingredient_mapper[j]
        encoded_ingr[index] = 1
    encoded_ingredients_list_test.append(encoded_ingr)

In [13]:
# Convert to Numpy Array

ing_train = [np.asarray(i) for i in encoded_ingredients_list_train]
cuis_train = [np.asarray(i) for i in encoded_cuisines_list_train]
ing_test = [np.asarray(i) for i in encoded_ingredients_list_test]

In [14]:
print len(encoded_ingredients_list_train)
print len(encoded_cuisines_list_train)
print len(encoded_ingredients_list_test)
print len(ing_train)
print len(cuis_train)
print len(ing_test)
print len(ing_train[1])

39774
39774
9944
39774
39774
9944
7126


## TensorFlow

In [29]:
import tensorflow as tf

## Construction Phase

* Must declare variables.
* We actually just defining things, no computation is performed.
* Constants, variables that only pass output and don't have any input.

* Placeholder variables are for input

<img src="diag.png" width="50%">

In [30]:
x = tf.placeholder(tf.float32, [None, 7126])

* Variables are for internal computation
* Store all the computational information

In [31]:
W = tf.Variable(tf.zeros([7126, 20]))
b = tf.Variable(tf.zeros([20]))

<img src="files/comp.png" width="60%">

In [32]:
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 20])

Loss function is defined as cross_entropy.

### Cross Entropy

In [33]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

* Determine parameters through back propagation
* Define the learning steps to minimize the loss by optimizing the learning rate of 0.01 Gradient Descent method.
* Backpropagation with Gradient Descent Algorithm

### Gradient Descent

In [34]:
# Learning Step
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [35]:
keep_prob = tf.placeholder("float")

## Execution Phase

In [38]:
# Initiates a session to actually perform computations
# Parralelization. We can assign CPUs/GPUs here
sess = tf.InteractiveSession()
# Initiate our variables
init = tf.initialize_all_variables()
sess.run(init)

In [37]:
# Close the session if running and recomputation is needed
# sess.close()

* We we actually training model
* Build model and slowly adding sets of K random observations
* Play around with number of passes and batch sizes

Neural network does not follow a linear path. Information is processed collectively.

In [ ]:
for i in range(4000):
    K = 30
    indices = random.sample(range(len(ing_train)), K)
    batch_xs = [ing_train[i] for i in sorted(indices)]
    batch_ys = [cuis_train[i] for i in sorted(indices)]
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

Iterations: 1000, K = 100, Training Accuracy: 75%

Iterations: 1000, K = 150, Training Accuracy: 76.3%

Iterations: 2000, K = 100, Training Accuracy: 79%

Iterations: 2000, K = 150, Training Accuracy: 81.2%

Iterations: 3000, K = 100, Training Accuracy: 82%

Iterations: 3000, K = 150, Training Accuracy: 81.1%

Iterations: 4000, K = 100, Training Accuracy: 80.7

Iterations: 4000, K = 150, Training Accuracy: 84.4%

Iterations: 4000, K = 50, Training Accuracy: 85.0%

Iterations: 4000, K = 30, Training Accuracy: 85.5% Testing Accuracy: 0.77484

Iterations: 5000, K = 100, Training Accuracy: 81.5

Iterations: 5000, K = 50, Training Accuracy: 86.4%

Iterations: 5000, K = 50, Training Accuracy: 86.4%

Iterations: 6000, K = 100, Training Accuracy: 85.5

Iterations: 6000, K = 100, Training Accuracy: 84 Testing Accuracy: 0.77233

Iterations: 10000, K = 50, Training Accuracy: 81.5 Testing Accuracy: 0.76478

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

Run a separate instance of interactive session for each computation.

In [ ]:
print sess.run(accuracy, feed_dict={x: ing_train, y_: cuis_train})

Retrieve prediction for test.

In [ ]:
prediction = tf.argmax(y,1).eval(feed_dict={x: ing_test, keep_prob: 1.0})

In [ ]:
cuis_pred = list()
for i in prediction:
    for name, num in cuisines_mapper.iteritems():
        if num == i:
            cuis_pred.append(name)

In [ ]:
import pandas as pd
dictionary = dict(zip(test_ids, cuis_pred))

df = pd.DataFrame(dictionary.items())
df.columns = ['id', 'cuisine']
df.head()

In [ ]:
df.to_csv("submission.csv", index = False)

# Results

<img src="top.png">

<img src="me.png">